## Imports

In [19]:
import pandas as pd
import json
from datetime import *

## Loading the data

In [2]:
# Loading sleep data from Oura json file available on https://cloud.ouraring.com/profile 
with open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_sleep_2022-12-06T10-07-33.json') as f:
    data1 = json.load(f)

sleep_df = pd.json_normalize(data1['sleep'], max_level=2)
print('sleep_df', sleep_df.shape)

sleep_df (176, 56)


In [3]:
# Loading raw wake up data from Google Form csv
wakeup_df = pd.read_csv ('questionnaire_data/raw/Wakeup_3_v1_raw.csv')
print('wakeup_df', wakeup_df.shape)

# Loading raw bedtime data from Google Form csv
bedtime_df = pd.read_csv ('questionnaire_data/raw/Bedtime_3_v1_raw.csv')
bedtime_df.columns = ['Timestamp_bedtime' if x=='Timestamp' else x for x in bedtime_df.columns]
print('bedtime_df', bedtime_df.shape)

wakeup_df (151, 9)
bedtime_df (149, 10)


In [22]:
print(sleep_df.iloc[0]['day'])

print(wakeup_df.loc[0]['Timestamp'])

2022-06-09
6/19/2022 8:01:31


## Combining the three dataframes

In [37]:
sleep_start_date = sleep_df.iloc[0]['day']
sleep_end_date = sleep_df.iloc[-1]['day']

wakeup_start_date = wakeup_df.loc[0]['Timestamp']
wakeup_end_date = wakeup_df.iloc[-1]['Timestamp']

bedtime_start_date = bedtime_df.loc[0]['Timestamp_bedtime']
bedtime_end_date = bedtime_df.iloc[-1]['Timestamp_bedtime']

def find_missing_days(df, start_date, end_date):
    return df.reindex(pd.date_range(start_date, end_date)).isnull().all(1)

missing_sleep_days = find_missing_days(sleep_df, sleep_start_date, sleep_end_date)
missing_wakeup_days = find_missing_days(wakeup_df, wakeup_start_date, wakeup_end_date)
missing_bedtime_days = find_missing_days(bedtime_df, bedtime_start_date, bedtime_end_date)

def remove_hour(string_date):
    split_string = string_date.split(" ")
    return split_string[0]

def compare_dates(sleep_date, wakeup_date, bedtime_date, earliest):
    y1, m1, d1 = [int(x) for x in sleep_date.split('-')]
    date1 = date(y1, m1, d1)

    m2, d2, y2 = [int(x) for x in remove_hour(wakeup_date).split('/')]
    date2 = date(y2, m2, d2)

    m3, d3, y3 = [int(x) for x in remove_hour(bedtime_date).split('/')]
    date3 = date(y3, m3, d3)

    if date1 == date2:
        earliest_date = latest_date = date1
    elif date1 > date2:
        earliest_date = date2
        latest_date = date1  
    else:
        earliest_date = date1
        latest_date = date2
    
    if earliest_date > date3:
        earliest_date = date3
    if latest_date < date3:
        latest_date = date3

    if earliest:
        return earliest_date
    else:
        return latest_date

latest_start = compare_dates(sleep_start_date, wakeup_start_date, bedtime_start_date, earliest = False)

earliest_end = compare_dates(sleep_end_date, wakeup_end_date, bedtime_end_date, earliest = True)


In [38]:
print(earliest_end)

2022-11-19


In [4]:
df = pd.concat([sleep_df, wakeup_df, bedtime_df])

df

,bedtime_end,bedtime_start,day,period,segment_state,time_in_bed,type,sleep_phase_5_min,restless,timezone,...,Timestamp_bedtime,"If you had coffee today, at what time did you take your last cup of coffee?","If you had alcohol today, at what time did you have your last drink?","If you did a workout today, at what time did you do your last workout?","If you did a stress-relieving activity today (meditation, yoga, etc), at what time did you do your last one?",Did you take medication that might affect your sleep?,How much did you eat in the last 3 hours before going to bed?,How would you characterize the activities you did in the last 3 hours before going to bed?,How much stress and/or anxiety do you feel now?,Is there any particular reason why you think you might or might not sleep well tonight?
0,2022-06-09T07:03:09+02:00,2022-06-08T23:01:09+02:00,2022-06-09,0.0,active,28920.0,rest,None,NaN,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-10T06:55:22+02:00,2022-06-09T22:46:22+02:00,2022-06-10,0.0,active,29340.0,long_sleep,4244222111111111112211111112233333222211112112...,6.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-11T06:50:26+02:00,2022-06-10T22:43:26+02:00,2022-06-11,0.0,active,29220.0,long_sleep,4422211111211111111111111111333322111222222222...,6.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-12T06:51:55+02:00,2022-06-11T23:14:55+02:00,2022-06-12,0.0,active,27420.0,long_sleep,4222221111111111121221112222333332222221222221...,7.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-13T07:21:57+02:00,2022-06-12T23:48:57+02:00,2022-06-13,0.0,active,27180.0,long_sleep,4422222122221111111122222111111123332222221112...,7.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/14/2022 23:14:11,NaN,NaN,NaN,NaN,No,0.0,Relaxed,1.0,NaN
145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/15/2022 22:55:39,NaN,NaN,10:30:00 AM,NaN,No,0.0,Relaxed,0.0,NaN
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/17/2022 2:10:08,NaN,1:00:00 AM,8:00:00 PM,NaN,No,0.0,"Stressful, Physically active, Socially active",2.0,NaN
147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/17/2022 22:30:21,NaN,NaN,NaN,NaN,No,0.0,Relaxed,1.0,NaN
